<a href="https://colab.research.google.com/github/mehdi-or/Variational-AE/blob/master/Mnist_AE%2Bclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)
import IPython as IP
IP.get_ipython().magic('reset -f')
import numpy as np
from keras.datasets import mnist
from keras import optimizers
from keras.layers import Input, Dense, concatenate, Dropout, Lambda
from keras.utils import to_categorical
#import pydotplus
#from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
from keras.models import Model
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
import os
from scipy import stats
from keras import backend as K
from keras.regularizers import l2, l1
#from keras.wrappers.scikit_learn import KerasClassifier
#from sklearn.model_selection import GridSearchCV
#keras.utils.vis_utils.pydotplus = pydotplus
from keras.utils.vis_utils import plot_model
start = time.time()
#%reset -f

DNN architechture

In [ ]:
encoding_dim = 2
drop = 0.01
activation_BT = 'linear'
num_epoch = 30
lr = 1e-4
discription = [str(encoding_dim), activation_BT, str(num_epoch), str(lr)]
def nna(encoding_dim):
      # this is our input placeholder for VT (creating a tensor)
      decoding_vox = Input(shape=(X_train.shape[1],), name="VT-input")

      encoded = Dense(512, activation='tanh', name="encoded_VT")(decoding_vox)
      encoded= Dropout(drop)(encoded)
      
      Bneck = Dense(encoding_dim, activation=activation_BT, name="Bneck")(encoded)
      Bneck.trainable = False
      classifier_stop = Lambda(lambda Bneck: Bneck)(Bneck)
      classifier_stop.trainable = False
      classifier_stop = Dense(10, activation='softmax', name='classifier')(Bneck)
      classifier_stop.trainable = False
        
      decoded = Dense(512, activation='tanh', name="decoded-VT")(Bneck)
      decoded= Dropout(drop)(decoded)
      
      decoded = Dense(X_train.shape[1], activation='linear', name="decoded-output")(decoded)
      decoded= Dropout(drop)(decoded)

      encoder = Model(decoding_vox, Bneck)
      decoder = Model(decoding_vox, [decoded, classifier_stop]) # model architecture
      plot_model(decoder, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
      
      
      bn_model = Model(inputs= decoder.input, outputs = decoder.get_layer('Bneck').output)
      
      # fitting the model and defining loss functiojn for each imput
      opt = optimizers.Adam(lr)
      decoder.compile(opt, loss=['mean_squared_error', 'categorical_crossentropy'],loss_weights=[1,1], metrics=['mse','accuracy'])
      return decoder, encoder, Bneck, bn_model
  

importing data

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = np.reshape(X_train/255,(X_train.shape[0],X_train.shape[1]**2))
X_test = np.reshape(X_test/255,(X_test.shape[0],X_test.shape[1]**2))

y_train0 = to_categorical(y_train, num_classes=10)
y_test0 = to_categorical(y_test, num_classes=10)

decoder, encoder, Bneck, bn_model = nna(encoding_dim)
decoder.save_weights('model.h5') #saving the initial weights right after compiling (for kfol CV)
decoder.fit(X_train, [X_train, y_train0], epochs= num_epoch, shuffle=True, validation_data=(X_test,[X_test, y_test0]), verbose=0)
intermediate_layer_model = Model(inputs=decoder.input, outputs=decoder.get_layer('Bneck').output)
BottleNeck = intermediate_layer_model.predict(X_test)
bt_features = bn_model.predict(X_test)
[decoded_VT, categ] = decoder.predict(X_test)
decoder.load_weights('model.h5') #after training, "reset" the model by reloading the initial weights (for kfol CV)
s=int(np.sqrt(decoded_VT.shape[1]))
decoded_VT0=np.reshape(decoded_VT,(decoded_VT.shape[0],s,s))
plt.imshow(decoded_VT0[0,:,:])
X_test = np.reshape(X_test,(X_test.shape[0],s,s))
plt.imshow(decoded_VT0[0,:,:])
plt.figure(2)
plt.imshow(X_test[0,:,:])

In [ ]:
plt.imshow(decoded_VT0[23,:,:])
plt.figure(2)
plt.imshow(X_test[23,:,:])

In [ ]:
scl = MinMaxScaler(feature_range=(0,1))
BN_feat_scl = scl.fit_transform(BottleNeck)

classifier

In [ ]:
'''
classes = np.unique(y_test)
class_number = len(classes)
labels2categ = to_categorical(y_test, num_classes = 10)
'''
scl = MinMaxScaler(feature_range=(0,1))
BN_feat_scl = scl.fit_transform(BottleNeck)

BN_feat = Input(shape=(BN_feat_scl.shape[1],), name="BN")
out_class = Dense(10, activation='softmax', name ="output", kernel_regularizer=l2(0.01))(BN_feat)
classifier = Model(BN_feat, out_class, name='classifier')
#chossing the learning rate is very important
opt = optimizers.Adam(lr = 5e-5)
classifier.compile(opt, loss = 'categorical_crossentropy',  metrics =['accuracy'])

#labels2categ = to_categorical(labels2num[0, :], num_classes=len(class_number))
history=classifier.fit(BottleNeck, y_test0, epochs=100, shuffle=True, validation_split = 0.2, verbose=0)

Plotting

In [ ]:
plt.figure(1)
plt.plot(np.asarray(history.history['val_accuracy']))
plt.plot(np.asarray(history.history['accuracy']))
plt.gca().legend(('val_accuracy_decoded','training_accuracy_decoded'), prop={'size': 12})
plt.xlabel('num_epochs')
plt.ylabel('accuracy')
plt.title('BN features')

plt.figure(2)
plt.plot(np.asarray(history.history['val_loss']))
plt.plot(np.asarray(history.history['loss']))
plt.gca().legend(('val_loss_decoded','training_loss_decoded'), prop={'size': 12})
plt.xlabel('no_epochs')
plt.ylabel('loss')
plt.title('BN features')

In [ ]:
#ax = plt.axes(projection='3d')
comp1 = BottleNeck[:,0]
comp2 = BottleNeck[:,1]
#comp3 = BottleNeck[:,2]
cmap = plt.cm.get_cmap('tab10')
#plt.scatter3D(comp1, comp2, comp3, c= y_test, cmap=cmap)
plt.scatter(comp1, comp2, c= y_test, cmap=cmap, alpha=.5)
plt.colorbar(ticks=range(10))

In [ ]:
#saving the parameters about the model
description = [str(encoding_dim), activation_BT, str(num_epoch), str(lr)]
pd.DataFrame.to_csv(description, ,index=['no_BT', 'activation', 'no_epochs', 'lr'])
